In [ ]:
%matplotlib inline

import matplotlib.pyplot as pyplot
from mpl_toolkits.mplot3d import Axes3D
import os
import sys
import numpy as np
import json
Arr = np.asarray

In [ ]:
from IPython.display import display, display_html
from IPython.display import HTML

In [ ]:
import vtk
import scipy.interpolate

In [ ]:
#https://stackoverflow.com/questions/4116658/faster-numpy-cartesian-to-spherical-coordinate-conversion
def appendSpherical_np(xyz):
    ptsnew = np.hstack((xyz, np.zeros(xyz.shape)))
    xy = xyz[:,0]**2 + xyz[:,1]**2
    ptsnew[:,3] = np.sqrt(xy + xyz[:,2]**2)
    ptsnew[:,4] = np.arctan2(np.sqrt(xy), xyz[:,2]) # for elevation angle defined from Z-axis down
    #ptsnew[:,4] = np.arctan2(xyz[:,2], np.sqrt(xy)) # for elevation angle defined from XY-plane up
    ptsnew[:,5] = np.arctan2(xyz[:,1], xyz[:,0])
    return ptsnew

In [ ]:
def json_samples_to_arrays(js):
    coordinates = []
    probabilities = []
    values = []
    cosn = []
    weights = []
    for smpl in js:
        probabilities.append(smpl['pdf'])
        coordinates.append(smpl['coord'])
        values.append(smpl['value'])
        cosn.append(smpl['cosn'])
        weights.append(smpl['weight'])
    coordinates = Arr(coordinates)
    probabilities = Arr(probabilities)
    values = Arr(values)
    cosn = Arr(cosn)
    weights = Arr(weights)
    return coordinates, probabilities, values, cosn, weights

In [ ]:
def json_test_cubemap_to_array(js):
    num_bins = len(js[0])
    idx = np.empty((6, num_bins, num_bins), np.int32)
    center = np.empty((6, num_bins, num_bins, 3), np.float32)
    corners = np.empty((6, num_bins, num_bins, 2, 3), np.float32)
    J = np.empty((6, num_bins, num_bins), np.float32)
    for side_idx, side in enumerate(js):
        for irow, row in enumerate(side):
            for icol, col in enumerate(row):
                cell = col
                idx[side_idx,irow,icol] = cell['index']
                corners[side_idx,irow,icol,0,:] = cell['w00']
                corners[side_idx,irow,icol,1,:] = cell['w11']
                center[side_idx,irow,icol,:] = cell['center']
                J[side_idx,irow,icol] = cell['J']
    return idx, center, corners, J

def to_cubemap_shape(a, idx):
    return a[idx.ravel(), ...].reshape(idx.shape+a.shape[1:])

def json_test_cubemap_to_arrays(js):
    num_bins = len(js[0])
    pr = np.empty((6, num_bins, num_bins))
    val = np.empty((6, num_bins, num_bins, 3))
    cnt = np.empty((6, num_bins, num_bins))
    for side_idx, side in enumerate(js):
        for irow, row in enumerate(side):
            for icol, col in enumerate(row):
                cell = col
                pr[side_idx,irow,icol] = cell['prob']
                cnt[side_idx,irow,icol] = float(cell['cnt'])
    return pr, cnt

def json_vis_cubemap_to_arrays(js):
    num_bins = len(js[0])
    pdf = np.empty((6, num_bins, num_bins))
    val = np.empty((6, num_bins, num_bins, 3))
    pos = np.empty((6, num_bins, num_bins, 3))
    for side_idx, side in enumerate(js):
        for irow, row in enumerate(side):
            for icol, col in enumerate(row):
                cell = col
                pdf[side_idx,irow,icol] = cell['pdf']
                val[side_idx,irow,icol,:] = cell['val']
                pos[side_idx,irow,icol] = cell['pos']
    return pos, val, pdf


In [ ]:
side_labels = '+X -X +Y -Y +Z -Z'.split()

In [ ]:
def plotcubemap(data, vmin=None, vmax=None):
    fig, axes = pyplot.subplots(2,3)
    axes = axes.flatten()
    axes = axes[[0, 3, 1, 4, 2, 5]]
    for side, ax, i in zip(range(6), axes, range(6)):
        ax.imshow(data[side,...], vmin=vmin, vmax=vmax, interpolation='none')
        ax.set(title = side_labels[side]+' '+str(i))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    return fig, axes

def plotcubemappolar(centers, data, vmin=None, vmax=None):
    polar = appendSpherical_np(centers.reshape((-1,3)))[:,[4,5]]
    data  = data.ravel()
    grid_x, grid_y = np.meshgrid(np.linspace(0., np.pi, centers.shape[1]*5), np.linspace(-np.pi, np.pi , centers.shape[1]*10))
    gridded_data = scipy.interpolate.griddata(polar, data, (grid_x, grid_y), method='linear', fill_value = vmin)
    #print (np.amin(polar, axis=0), np.amax(polar, axis=0))
    fig, ax = pyplot.subplots(1,1)
    ax.imshow(gridded_data.T)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    return fig, ax

with open('/tmp/debug2.json', 'r') as f:
    all_data = json.load(f)
# all_data is deserialized into python types!
coordinates, probabilities, values, cosn, weights = json_samples_to_arrays(all_data['vis_samples'])
values = np.average(values, axis=1)
weights = np.average(weights, axis=1)

spherecoord = appendSpherical_np(coordinates)[:,3:]
spherecoord[:,[1,2]] *= 180./np.pi

if 1:
    cube_probabilities, cube_counts = json_test_cubemap_to_arrays(all_data['test_cubemap'])
    cube_counts /= np.sum(cube_counts)

    tc_pos, tc_val, tc_pdf = json_vis_cubemap_to_arrays(all_data['vis_cubemap'])
    tc_val = np.average(tc_val, axis=3)

In [ ]:
def plot_cubemap_ndf_viz(filename, polar = True):
    display(HTML('<h1>'+filename+'</h1>'))
    with open(filename, 'r') as f:
        all_data = json.load(f)
    idx, centers, _, J = json_test_cubemap_to_array(all_data['cubemap'])
    probs = to_cubemap_shape(np.asarray(all_data['probs'], np.float32), idx)
    probs /= J
    counts = to_cubemap_shape(np.asarray(all_data['counts'], np.float32), idx)
    counts /= J
    counts /= np.sum(counts)

    data = np.log(probs+1.e-4) 
    
    if polar:
        fig, _ = plotcubemappolar(centers, data, np.log(1.e-4), 0.)
    else:
        fig, _ = plotcubemap(data, np.log(1.e-4), 0.)
    fig.suptitle('probs')
    display(fig)
    
    data = np.log(counts+1.e-4)
    if polar:
        fig, _ = plotcubemappolar(centers, data, np.log(1.e-4), 0.)
    else:
        fig, _ = plotcubemap(data, np.log(1.e-4), 0.)
    fig.suptitle('counts')
    display(fig)

In [ ]:
pyplot.ioff()
if 1:
    for roughness in ['rough', 'sharp']:
        for ior in ['D-L', 'L-D']:
            for wi in ['below','vertical', 'deflected', '45']:
                plot_cubemap_ndf_viz('/tmp/cubemap_%s_%s_%s.json' % (wi, roughness, ior))
pyplot.close('all') 
# plot_cubemap_ndf_viz('/tmp/cubemap_vertical_sharp_D-L.json')
# plot_cubemap_ndf_viz('/tmp/cubemap_deflected_sharp.json')
# plot_cubemap_ndf_viz('/tmp/cubemap_45_sharp.json')
# plot_cubemap_ndf_viz('/tmp/cubemap_below_rough.json')
# plot_cubemap_ndf_viz('/tmp/cubemap_vertical_rough.json')
#plot_cubemap_ndf_viz('/tmp/cubemap_deflected_rough.json')
#plot_cubemap_ndf_viz('/tmp/cubemap_45_rough_D-L.json', polar=False)

In [ ]:
def MakeLUTFromCTF():
    """
    Use a color transfer Function to generate the colors in the lookup table.
    See: http://www.vtk.org/doc/nightly/html/classvtkColorTransferFunction.html
    :param: tableSize - The table size
    :return: The lookup table.
    """
    ctf = vtk.vtkColorTransferFunction()
    ctf.SetColorSpaceToDiverging()
    ctf.AddRGBPoint(0.0, 0.0, 0.5, 1)
    ctf.AddRGBPoint(0.5, 0.8, 0.2, 1)
    ctf.AddRGBPoint(1.0, 0.6, 0.0, 1)
    return ctf

In [ ]:
fig = pyplot.figure()
ax = fig.add_subplot(131, projection='polar')
ax.scatter(spherecoord.T[1]/180.*np.pi, values)
ax = fig.add_subplot(132, projection='polar')
ax.scatter(spherecoord.T[1]/180.*np.pi, probabilities)
ax = fig.add_subplot(133, projection='polar')
ax.scatter(spherecoord.T[1]/180.*np.pi, weights)

In [ ]:
fig = pyplot.figure()
ax = fig.add_subplot(121, projection='3d')
ax.scatter(coordinates[:,0], coordinates[:,1], coordinates[:,2], c=probabilities)
ax = fig.add_subplot(122, projection='3d')
ax.scatter(coordinates[:,0], coordinates[:,1], coordinates[:,2], c=values)

In [ ]:
fig = pyplot.figure()
ax = fig.add_subplot(111)
#ax.tripcolor(spherecoord[:,1], spherecoord[:,2], np.log(probabilities))
ax.scatter(spherecoord[:,1], spherecoord[:,2], s=20., c = probabilities, edgecolor='None')

In [ ]:
data = np.log(cube_probabilities+1.e-4)
fig, _ = plotcubemap(data, np.log(1.e-4), 0.)
fig.suptitle('probs')
data = np.log(cube_counts+1.e-4)
fig, _ = plotcubemap(data, np.log(1.e-4), 0.)
fig.suptitle('counts')

In [ ]:
data = np.log(tc_val+1.e-4)
fig, _ = plotcubemap(data, np.log(1.e-4), 0.)
fig.suptitle('value')
data = np.log(tc_pdf+1.e-4)
fig, _ = plotcubemap(data, np.log(1.e-4), 0.)
fig.suptitle('pdf')